# 1) Loading corpus and queries

In [1]:
from datasets import load_dataset

ds = load_dataset("clarin-knext/fiqa-pl", "corpus")
ds2 = load_dataset("clarin-knext/fiqa-pl", "queries")
corpus = ds['corpus'].to_pandas()
queries = ds2['queries'].to_pandas()

In [2]:
queries.head()

,_id,title,text
0,0,,Co jest uważane za wydatek służbowy w podróży ...
1,4,,Wydatki służbowe - ubezpieczenie samochodu pod...
2,5,,Rozpoczęcie nowego biznesu online
3,6,,„Dzień roboczy” i „termin płatności” rachunków
4,7,,Nowy właściciel firmy – Jak działają podatki d...


# 2) Creating a dataset of positive and negative sentence pairs

First we will load the dataset that conects queries with corpus by id, question and answer.

In [3]:
ds3 = load_dataset("clarin-knext/fiqa-pl-qrels")
ds3

DatasetDict({
    train: Dataset({
        features: ['query-id', 'corpus-id', 'score'],
        num_rows: 14166
    })
    validation: Dataset({
        features: ['query-id', 'corpus-id', 'score'],
        num_rows: 1238
    })
    test: Dataset({
        features: ['query-id', 'corpus-id', 'score'],
        num_rows: 1706
    })
})

In [4]:
validation = ds3['validation'].to_pandas()
train = ds3['train'].to_pandas()
test = ds3['test'].to_pandas()

In [5]:
validation.head()

,query-id,corpus-id,score
0,1,14255,1
1,2,308938,1
2,3,296717,1
3,3,100764,1
4,3,314352,1


let's see the question and answer

In [6]:
print("Question: ", queries[queries['_id'] == '1']['text'].iloc[0])
print("Answer: ", corpus[corpus['_id'] == '14255']['text'].iloc[0])

Question:  Zgłaszanie wydatków biznesowych dla firmy bez dochodu
Answer:  Tak, możesz ubiegać się o potrącenia biznesowe, jeśli nie uzyskujesz jeszcze żadnych dochodów. Ale najpierw powinieneś zdecydować, jaką strukturę chcesz mieć dla swojej firmy. Albo struktura firmy, albo jednoosobowy przedsiębiorca lub spółka osobowa. Struktura firmy Jeśli wybierzesz strukturę firmy (która jest droższa w przygotowaniu), będziesz ubiegać się o potrącenia, ale bez dochodu. Więc ponosiłbyś stratę i kontynuowałbyś straty, dopóki dochód z firmy nie przekroczy twoich wydatków. Straty te pozostaną więc w firmie i będą mogły zostać przeniesione na przyszłe lata dochodowe, kiedy będziesz osiągać zyski, aby zrekompensować te zyski. Więcej informacji można znaleźć w ATO – Straty podatkowe firmy. Przedsiębiorca jednoosobowy o strukturze partnerskiej Jeśli zdecydujesz się być jednoosobowym przedsiębiorcą lub spółką osobową, a Twoja firma ponosi straty, musisz sprawdzić zasady dotyczące strat niekomercyjnych, a

Loading model

In [7]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch import nn
import torch

model_path = "clarin-knext/herbert-base-reranker-msmarco"
model = AutoModelForSequenceClassification.from_pretrained(
    model_path
    ).to("cuda")
model.classifier = nn.Linear(768, 2, device=model.device)
model.num_labels = 2
model.config.num_labels = 2


tokenizer = AutoTokenizer.from_pretrained(model_path)
features = tokenizer(['Jakie miasto jest stolica Polski?', 'Stolicą Polski jest Warszawa.'],  padding=True, truncation=True, return_tensors="pt").to("cuda")
model.eval()
with torch.no_grad():
    scores = model(**features).logits
    print(scores)

tensor([[0.1710, 0.8694],
        [0.1697, 0.8670]], device='cuda:0')


In [8]:
separator = tokenizer.sep_token
separator

'</s>'

Preparing dataset

In [9]:
import pandas as pd

big_df = pd.concat([validation,train,test]).reset_index(drop=True)

Positive sentences

In [10]:
list_ = []

for i in range(2):
  for index, row in big_df.iterrows():

    corpus_id = row['corpus-id']
    query_id = row['query-id']

    list_.append({
      'sentence': queries[queries['_id'] == str(query_id)]['text'].iloc[0] + separator + corpus[corpus['_id'] == str(corpus_id)]['text'].iloc[0],
      'label': 1
    })

list_[:5]

[{'sentence': 'Zgłaszanie wydatków biznesowych dla firmy bez dochodu</s>Tak, możesz ubiegać się o potrącenia biznesowe, jeśli nie uzyskujesz jeszcze żadnych dochodów. Ale najpierw powinieneś zdecydować, jaką strukturę chcesz mieć dla swojej firmy. Albo struktura firmy, albo jednoosobowy przedsiębiorca lub spółka osobowa. Struktura firmy Jeśli wybierzesz strukturę firmy (która jest droższa w przygotowaniu), będziesz ubiegać się o potrącenia, ale bez dochodu. Więc ponosiłbyś stratę i kontynuowałbyś straty, dopóki dochód z firmy nie przekroczy twoich wydatków. Straty te pozostaną więc w firmie i będą mogły zostać przeniesione na przyszłe lata dochodowe, kiedy będziesz osiągać zyski, aby zrekompensować te zyski. Więcej informacji można znaleźć w ATO – Straty podatkowe firmy. Przedsiębiorca jednoosobowy o strukturze partnerskiej Jeśli zdecydujesz się być jednoosobowym przedsiębiorcą lub spółką osobową, a Twoja firma ponosi straty, musisz sprawdzić zasady dotyczące strat niekomercyjnych, aby

Negative sentences

In [11]:
len(list_)

34220

In [12]:
for index, row in big_df.iterrows():

  corpus_id = row['corpus-id']
  query_id = row['query-id']

  for i in range(3):
    
    index = (corpus_id + i)%len(corpus)
    if corpus.iloc[index]['_id'] != corpus_id:
      list_.append({
        'sentence': queries[queries['_id'] == str(query_id)]['text'].iloc[0] + separator + corpus.iloc[index]['text'],
        'label': 0
      })

In [13]:
len(list_)

85550

In [14]:
from sklearn.model_selection import train_test_split
from datasets import Dataset

df = pd.DataFrame(list_, columns=['sentence', 'label'])

temp, test = train_test_split(df, test_size=0.2, random_state=1234)
train, eval = train_test_split(temp, test_size=0.25, random_state=1234)

train = Dataset.from_pandas(train)
train = train.class_encode_column("label")
test = Dataset.from_pandas(test)
test = test.class_encode_column("label")
eval = Dataset.from_pandas(eval)
eval = eval.class_encode_column("label")

train= train.rename_column("__index_level_0__", "input_ids")
test = test.rename_column("__index_level_0__", "input_ids")
eval = eval.rename_column("__index_level_0__", "input_ids")

Stringifying the column:   0%|          | 0/51330 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/51330 [00:00<?, ? examples/s]

Stringifying the column:   0%|          | 0/17110 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/17110 [00:00<?, ? examples/s]

Stringifying the column:   0%|          | 0/17110 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/17110 [00:00<?, ? examples/s]

In [15]:
print("Train length: ", len(train), " Test length: ", len(test), " Evaluation: ", len(eval))

Train length:  51330  Test length:  17110  Evaluation:  17110


In [16]:
train[0]

{'sentence': 'Jak sprawdzić, czy firma jest legalna?</s>Jeśli masz pieniądze i determinację, aby spłacić wszystkie karty w ciągu sześciu miesięcy, zamówienie nie wpłynie na twoją zdolność kredytową i finanse. Gdybyś miał mniej dostępnych pieniędzy (powiedzmy, że mógłbyś spłacić łącznie 500 USD miesięcznie), wtedy ze względów finansowych dobrze byłoby spłacić najpierw kartę kredytową o najwyższym oprocentowaniu, więc płacisz mniejsze odsetki. Byłoby dobrze ze względów psychologicznych, aby najpierw zapłacić kartą najmniejszą kwotą (aby szybko poczuć sukces, a niektórzy ludzie potrzebują tego poczucia sukcesu, aby dalej się opłacać, tylko psychologicznie). A jeśli te rzeczy są ze sobą sprzeczne, zastanów się, co jest ważniejsze. I cokolwiek robisz, spłacanie długu jest lepsze niż niespłacanie go. Więc jeśli nie możesz się zdecydować, płacisz #1, potem #2, potem #3, potem #4, a potem #5.',
 'label': 0,
 'input_ids': 53483}

In [17]:
from transformers import Trainer, TrainingArguments
import numpy as np

# define training arguments
training_args = TrainingArguments(
    output_dir="training",
    num_train_epochs=3,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_eval_batch_size=24,
    per_device_train_batch_size=24
)

def compute_metric(eval_prediction):
    predictions, labels = eval_prediction
    predictions = np.argmax(predictions, axis=-1)
    accuracy = np.mean(predictions == labels)
    return {"accuracy": accuracy}

d:\programowanie\PJN\Natural_Language_Processing_Labs\.venv\Lib\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [18]:
def tokenizer_fn(batch):
    return tokenizer(batch['sentence'], truncation=True, padding=True)

tokenized_train = train.map(tokenizer_fn, batched=True)
tokenized_eval = eval.map(tokenizer_fn, batched=True)
tokenized_test = test.map(tokenizer_fn, batched=True)

Map:   0%|          | 0/51330 [00:00<?, ? examples/s]

Map:   0%|          | 0/17110 [00:00<?, ? examples/s]

Map:   0%|          | 0/17110 [00:00<?, ? examples/s]

Custom trainer with weighted classes

In [21]:
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch = None):
        labels = inputs.pop("labels")
        # forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")
        # compute custom loss (suppose one has 2 labels with different weights)
        loss_fct = nn.CrossEntropyLoss(weight=torch.tensor([1.0, 5.0], device=model.device))
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

In [22]:
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    tokenizer=tokenizer,
    compute_metrics=compute_metric
)

C:\Users\User\AppData\Local\Temp\ipykernel_23208\3114532008.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `CustomTrainer.__init__`. Use `processing_class` instead.
  trainer = CustomTrainer(


In [23]:
trainer.train(resume_from_checkpoint = True)

  0%|          | 0/6417 [00:00<?, ?it/s]

{'loss': 0.2812, 'grad_norm': 2.461892604827881, 'learning_rate': 4.610409848839022e-05, 'epoch': 0.23}
{'loss': 0.2245, 'grad_norm': 2.900048017501831, 'learning_rate': 4.220819697678043e-05, 'epoch': 0.47}
{'loss': 0.1968, 'grad_norm': 21.29344940185547, 'learning_rate': 3.831229546517064e-05, 'epoch': 0.7}
{'loss': 0.2121, 'grad_norm': 6.034542560577393, 'learning_rate': 3.441639395356086e-05, 'epoch': 0.94}


  0%|          | 0/713 [00:00<?, ?it/s]

{'eval_loss': 0.22107411921024323, 'eval_accuracy': 0.9330800701344243, 'eval_runtime': 335.9443, 'eval_samples_per_second': 50.931, 'eval_steps_per_second': 2.122, 'epoch': 1.0}
{'loss': 0.1731, 'grad_norm': 5.430860996246338, 'learning_rate': 3.0520492441951073e-05, 'epoch': 1.17}
{'loss': 0.1521, 'grad_norm': 14.77031135559082, 'learning_rate': 2.6624590930341282e-05, 'epoch': 1.4}
{'loss': 0.138, 'grad_norm': 40.71648025512695, 'learning_rate': 2.2728689418731498e-05, 'epoch': 1.64}
{'loss': 0.1357, 'grad_norm': 5.493531227111816, 'learning_rate': 1.883278790712171e-05, 'epoch': 1.87}


  0%|          | 0/713 [00:00<?, ?it/s]

{'eval_loss': 0.18507573008537292, 'eval_accuracy': 0.9457627118644067, 'eval_runtime': 345.6799, 'eval_samples_per_second': 49.497, 'eval_steps_per_second': 2.063, 'epoch': 2.0}


KeyboardInterrupt: 

In [22]:
tokenized_eval

Dataset({
    features: ['sentence', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 13688
})